In [ ]:
# Our lovely import statements :)

import pandas as pd

import os
import glob
import ntpath

from PIL import Image
import cv2
import matplotlib.pyplot as plt
from skimage import io

from keras_preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# putting all image paths of each dataset together

test_map = [] 
train_map =[]
valid_map = []

for sub_dir_path in glob.glob("data/MMAFEDB/test/"+"*"):
    try:
        dir_name = sub_dir_path.split('/')[-1]
        for filename in os.listdir(sub_dir_path):
            image_path = sub_dir_path + '/' + filename
            test_map.extend([dir_name, image_path])
    except Exception as e:
        print(e)

for sub_dir_path in glob.glob("data/MMAFEDB/train/"+"*"):
    try:
        dir_name = sub_dir_path.split('/')[-1]
        for filename in os.listdir(sub_dir_path):
            image_path = sub_dir_path + '/' + filename
            train_map.extend([dir_name, image_path])
    except Exception as e:
        print(e)
        
for sub_dir_path in glob.glob("data/MMAFEDB/valid/"+"*"):
    try:
        dir_name = sub_dir_path.split('/')[-1]
        for filename in os.listdir(sub_dir_path):
            image_path = sub_dir_path + '/' + filename
            valid_map.extend([dir_name, image_path])
    except Exception as e:
        print(e)

In [ ]:
# creating dataframes with labels of 4 emotions used 

df_test = pd.DataFrame({"path" : test_map[1::2]})
df_train = pd.DataFrame({"path" : train_map[1::2]})
df_valid = pd.DataFrame({"path" : valid_map[1::2]})

def labels(path):
    pat = os.path.dirname(path) 
    return ntpath.basename(pat)
df_test['label'] = df_test['path'].apply(lambda x: labels(x))
df_train['label'] = df_train['path'].apply(lambda x: labels(x))
df_valid['label'] = df_valid['path'].apply(lambda x: labels(x))

lst = ['disgust','surprise','fear']

df_test = df_test[(df_test.label != 'disgust') & (df_test.label != 'surprise') & (df_test.label != 'fear')]
df_train = df_train[(df_train.label != 'disgust') & (df_train.label != 'surprise') & (df_train.label != 'fear')]
df_valid = df_valid[(df_valid.label != 'disgust') & (df_valid.label != 'surprise') & (df_valid.label != 'fear')]


In [ ]:
# Using IDG to make all the images the same size and to create more images for our data to train and validate with

idg = ImageDataGenerator(rescale=1/255,     
                            rotation_range=15,
                            width_shift_range=0.15,
                            height_shift_range=0.15,
                            shear_range=0.15,
                            zoom_range=0.15,
                            horizontal_flip=True)

train_gen = idg.flow_from_dataframe(dataframe=df_train, directory= './', x_col='path', y_col='label', 
                                    class_mode="categorical", target_size = [48,48], color_mode = 'grayscale')

valid_gen = idg.flow_from_dataframe(dataframe=df_valid, directory= './', x_col='path', y_col='label', 
                                   class_mode="categorical", target_size = [48,48], color_mode = 'grayscale')

In [ ]:
# Created CNN based off of AlexNet, but it's altered more 
# in order for the program to detect the emotions better from the input

best = Sequential([
    layers.Conv2D(128, 6, activation="relu", padding="same", input_shape=[48,48,1]),#input shape: [rows, columns, channels]
    layers.MaxPooling2D(4),
    layers.Conv2D(128, 2, activation="relu", padding="same"),
    layers.Conv2D(256, 2, activation="relu", padding="same"),
    layers.MaxPooling2D(4),
    layers.Conv2D(128, 2, activation="relu", padding="same"),
    layers.Conv2D(64, 2, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dense(48, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(4, activation="softmax")
])


best.compile(loss='categorical_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

In [ ]:
mc = ModelCheckpoint('best_model.h5', 
                     save_best_only=True,
                     save_weights_only=False, 
                     save_freq='epoch')
es = EarlyStopping(monitor='val_accuracy', 
                   mode='auto', 
                   verbose=1, 
                   patience=15, 
                   restore_best_weights=True)
history = best.fit(train_gen,
                    epochs=500,
                    validation_data = valid_gen, 
                    callbacks = [es,mc])
score = best.evaluate(valid_gen, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])  # this is the one we care about